## Naive Bayes EDA

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline


In [2]:
df_train_var = pd.read_csv("../data/training_variants.txt")
df_test_var = pd.read_csv("../data/test_variants")
df_train_txt = pd.read_csv('/Users/wanggene/Downloads/MoreData/kaggle_P04/training_text.txt', sep='\|\|', header=None, skiprows=1,names=["ID","Text"])
df_test_txt = pd.read_csv('/Users/wanggene/Downloads/MoreData/kaggle_P04/test_text.txt', sep='\|\|', header=None, skiprows=1,names=["ID","Text"])


/Users/wanggene/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/wanggene/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


In [3]:
df_train_txt.columns

Index(['ID', 'Text'], dtype='object')

In [7]:
import re
s = "string$%. sd 0.5 m*l With. 0.5 ad P145A Punctuation?"
s = re.sub(r'[^\w\s]','',s)
s = re.sub(r'[^\D]', '', s)
s

'string sd  ml With  ad PA Punctuation'

In [15]:
import re
def remove_pun(text):
    s = re.sub(r'[^\w\s]','',text)
    s = re.sub(r'[^\D]', '', s)   # add new
    return s

df_train_txt['Text_cleaned'] = df_train_txt.Text.apply(remove_pun)
df_test_txt['Text_cleaned'] =  df_test_txt.Text.apply(remove_pun)


In [17]:
df_train_txt.head()

,ID,Text,Text_cleaned
0,0,Cyclin-dependent kinases (CDKs) regulate a var...,Cyclindependent kinases CDKs regulate a variet...
1,1,Abstract Background Non-small cell lung canc...,Abstract Background Nonsmall cell lung cance...
2,2,Abstract Background Non-small cell lung canc...,Abstract Background Nonsmall cell lung cance...
3,3,Recent evidence has demonstrated that acquired...,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...,Oncogenic mutations in the monomeric Casitas B...


In [20]:
df_train_txt.Text_cleaned[1][:1000]

' Abstract Background  Nonsmall cell lung cancer NSCLC is a heterogeneous group of disorders with a number of genetic and proteomic alterations cCBL is an E ubiquitin ligase and adaptor molecule important in normal homeostasis and cancer We determined the genetic variations of cCBL relationship to receptor tyrosine kinases EGFR and MET and functionality in NSCLC  Methods and Findings  Using archival formalinfixed paraffin embedded FFPE extracted genomic DNA we show that cCBL mutations occur in somatic fashion for lung cancers cCBL mutations were not mutually exclusive of MET or EGFR mutations however they were independent of p and KRAS mutations In normaltumor pairwise analysis there was significant loss of heterozygosity LOH for the cCBL locus  n\u200a\u200a and none of these samples revealed any mutation in the remaining copy of cCBL The cCBL LOH also positively correlated with EGFR and MET mutations observed in the same samples Using select cCBL somatic mutations such as SNHY QE and

### Merge the tables

In [12]:
df_train = pd.merge(df_train_var, df_train_txt, how='left', on='ID')
df_train = df_train[df_train.Text != '']
print(df_train.shape)

# test dataset
df_test = pd.merge(df_test_var, df_test_txt, how='left', on='ID')
#df_test.head()
df_train.head()

(3321, 5)


,ID,Gene,Variation,Class,Text
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,2,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,3,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...


In [24]:
# This cell reduces the training data for Kaggle limits. Remove this cell for real results.
from sklearn.model_selection import train_test_split
df_train, _ = train_test_split(df_train, test_size=0.7, random_state=8, stratify=df_train['Class'])
df_train.shape

(298, 5)

In [21]:
from sklearn.feature_extraction import text 
my_stopword = [",", '.', '(', ')', '%', ';', 'al.', ':', '[', ']', '!', '#', '$', '&',"*", "**", "'", "''"] # 
my_stop_words = text.ENGLISH_STOP_WORDS.union(my_stopword) #'english'

In [25]:
#### Concept of countvectorizer
# https://www.youtube.com/watch?v=bPYJi1E9xeM
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

X1_train_test = df_train['Text'][0:100]
count_vectorizer_test = CountVectorizer(
    analyzer="word", tokenizer=nltk.word_tokenize,
    preprocessor=None, stop_words=my_stop_words, max_features=None) 

bag_of_words_test = count_vectorizer_test.fit_transform(X1_train_test)
#a = bag_of_words_test.toarray()
#print(a[0])
#print(count_vectorizer.inverse_transform(a[0]))

In [ ]:
#### Concept of TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf_test = TfidfVectorizer(
    analyzer="word", tokenizer=nltk.word_tokenize,
    preprocessor=None, stop_words='english', max_features=None) 
bag_of_words_test = Tfidf_test.fit_transform(X1_train_test)
a = bag_of_words_test.toarray()
print(a[0])
#print(Tfidf_test.inverse_transform(a[0]))

In [31]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [32]:
X = df_train['Text'][0:100]
y = df_train['Class'].values.ravel()[0:100]

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk


count_vectorizer_test = CountVectorizer(
    analyzer="word", tokenizer=nltk.word_tokenize,
    preprocessor=None, stop_words=my_stop_words, max_features=None) 

X_train = count_vectorizer_test.fit_transform(X)


In [37]:
nb.fit(X_train, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [38]:
X_train_tokens = count_vectorizer_test.get_feature_names()
X_train_tokens[:20]

["'-thiotriphos-",
 "'activation",
 "'activator",
 "'classic",
 "'closed",
 "'common",
 "'de",
 "'discrepancy",
 "'effector",
 "'gdp'aifj",
 "'guardian",
 "'hot",
 "'initiator",
 "'kegg",
 "'large",
 "'mark",
 "'mutational",
 "'normal",
 "'off",
 "'on"]

In [39]:
X_train_tokens[600:650]

['.82–1.04',
 '.83',
 '.84',
 '.87',
 '.88',
 '.89',
 '.8b',
 '.8d',
 '.9',
 '.9,36,37',
 '.90',
 '.91',
 '.91–1.0',
 '.92',
 '.95',
 '.989',
 '.990',
 '.9g',
 '.9–4.1',
 '.alteration',
 '.although',
 '.as',
 '.because',
 '.c',
 '.clearly',
 '.coli',
 '.d-type',
 '.discussionsomatic',
 '.discussionusing',
 '.discussionwe',
 '.dna',
 '.drug',
 '.e',
 '.egfr',
 '.f',
 '.fig',
 '.figure',
 '.for',
 '.forty-five',
 '.gelatin',
 '.gray',
 '.here',
 '.however',
 '.human',
 '.identi¢cation',
 '.immunoblot',
 '.in',
 '.lvs',
 '.lvssrssqqtc',
 '.mammalian']

In [43]:
print(count_vectorizer_test.get_feature_names()[:10])
nb.feature_count_[:, :10]

["'-thiotriphos-", "'activation", "'activator", "'classic", "'closed", "'common", "'de", "'discrepancy", "'effector", "'gdp'aifj"]


array([[ 0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  0.,  2.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [44]:
class_1_token_count = nb.feature_count_[0,:]
class_2_token_count = nb.feature_count_[1,:]
class_3_token_count = nb.feature_count_[2,:]
class_4_token_count = nb.feature_count_[3,:]
class_5_token_count = nb.feature_count_[4,:]
class_6_token_count = nb.feature_count_[5,:]
class_7_token_count = nb.feature_count_[6,:]
class_8_token_count = nb.feature_count_[7,:]
class_9_token_count = nb.feature_count_[8,:]


In [45]:
df_token_count = pd.DataFrame({'token_name' : X_train_tokens,
                               'class1': class_1_token_count,
                               'class2': class_2_token_count,
                               'class3': class_3_token_count,
                               'class4': class_4_token_count,
                               'class5': class_5_token_count,
                               'class6': class_6_token_count,
                               'class7': class_7_token_count,
                               'class8': class_8_token_count,
                               'class9': class_9_token_count})
df_token_count.head()

,class1,class2,class3,class4,class5,class6,class7,class8,class9,token_name
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,'-thiotriphos-
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,'activation
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,'activator
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,'classic
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,'closed


In [46]:
df_token_count[12345:12365]

,class1,class2,class3,class4,class5,class6,class7,class8,class9,token_name
12345,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,cdk4-t172a
12346,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,cdk4-wt
12347,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,cdk4-y17f
12348,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,cdk4-δg
12349,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,cdk4.the
12350,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,cdk4/6
12351,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,cdk4/6-specific
12352,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,cdk4/6/cyclin
12353,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,cdk4/cdk6
12354,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,cdk4/cyclin


In [48]:
class_num = 'class3'
df_token_count.sort_values(class_num, ascending=False)[['token_name', class_num]][:10]

,token_name,class3
842,0,522.0
1249,1,345.0
7358,>,240.0
32781,rad51c,205.0
11818,cancer,195.0
17564,exon,170.0
28733,ovarian,157.0
26721,mutations,142.0
39699,variants,112.0
3067,2,106.0
